# NLP 1. Fake News and Hate Speech identifier

**James Morgan (jhmmorgan)**

_2022-05-04_

# 📖 Background

We want a proof of concept, where an end user can easily be provided with a summary of a news article, along with a warning on whether the text is likely to contain hate speech or fake news.

This proof of concept would be in the form of a standalone application that when provided the URL to a news article, provides the end-user with the summary of the article, along with a flag if the article may contain hate speech or fake news.

### The Task
This notebook is **Part 1** of my NLP project.  The task of this notebook is to create usable models that can help us predict if an article of text might contain either hate speech or fake news.

# 📚 Libraries and functions
We'll start by loading the libraries, the majority of which are sklearn modules

In [1]:
from utils import *
from nlp_models import *

import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.metrics import f1_score, accuracy_score

---
## Hate Speech

We want to create a hate speech prediction mode.  We can do this using a database of tweets that have already been labeled as either hate speech (1) or not hate speech (0).

In [2]:
hs_train = pd.read_csv("./data/hate speech/train.csv")

print2.bold("Training Set")
print(f"Shape: {hs_train.shape}, Len: {len(hs_train)}")
print()
print2.heading("Head of training set")
print(hs_train.head())
print()
print2.bold("Tweet No. 1")
print(hs_train.tweet.iloc[1])
print2.bold("Tweet No. 22768")
print(hs_train.tweet.iloc[22768])

Training Set
Shape: (31962, 3), Len: 31962

Head of training set
   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

Tweet No. 1
@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked
Tweet No. 22768
imagine how r founding fathers wldve handled muslim terrorism. contrast that w/the limp-dicked pc bullshit of the @user admin  


#### Building the model
We'll want to feed our custom model a pipeline that processes and classifies our tweets.

1. We want to use **CountVectorizer()** on our tweets.  This creates a matrix of 1's (appears) and 0's (doesn't appear) of one to two words.
2. We'll then apply a **TfidfTransformer()** to this matrix, which normalises it.  TF-IDF is an information retrieval and information extraction subtask which aims to express the importance of a word to a document which is part of a collection.
3. Finally, we'll apply the **SGDClassifier()**, to classify / predict each normalised matrix.

In [3]:
dhs_pipeline = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
                         ('tfidf',  TfidfTransformer()),
                         ('sgd', SGDClassifier()),])
dhs = nlp_model(dhs_pipeline)

We then want to split our data into training and testing data sets, to validate our results.  However, prior to doing this, we'll want to resample our data using our custom resample function.  This function will resample all minority labels to appear the same number of times as the majority label.

In other words, if out of a training set of 1000 tweets, 700 were not hate speech but 300 were, this would resample / reuse the 300 tweets, so that there was 700 each totalling 1400 tweets.  This helps prevent bias towards the majority label.

We then use this to fit (and if applicable, save) our model, before predicting the accuracy of our model.

In [4]:
hs_train = dhs.resample(hs_train, "label")
hs_X_train, hs_X_test, hs_y_train, hs_y_test = train_test_split(hs_train['tweet'], hs_train['label'], random_state = dhs.random_state)

In [5]:
dhs.fit(hs_X_train, hs_y_train)
#dhs.save_model("./models/hate_speech_v1.pkl")
#dhs = nlp_model()
#dhs.load_model("./models/hate_speech_v1.pkl")

In [6]:
hs_y_predict = dhs.predict(hs_X_test)
output_score(hs_y_test, hs_y_predict, [0, 1], f1_score)

filter_TP = ((hs_y_test == 1) & (hs_y_predict == 1))
filter_TN = ((hs_y_test == 0) & (hs_y_predict == 0))
filter_FP = ((hs_y_test == 0) & (hs_y_predict == 1))
filter_FN = ((hs_y_test == 1) & (hs_y_predict == 0))

print()
print2.heading("Samples of each prediction outcome")
print_samples(2, hs_X_test, filter_TP, title = "True Positive")
print_samples(2, hs_X_test, filter_TN, title = "True Negative")
print_samples(2, hs_X_test, filter_FP, title = "False Positive")
print_samples(2, hs_X_test, filter_FN, title = "False Negative")

Accuracy: 98.09%
Confusion Matrix
      0     1
0  7105   253
1    37  7465

Samples of each prediction outcome
True Positive
this idiot makes my blood boil!  #pig #liar 
seems like @user "#canadianvalues" are ignoring reality, dividing &amp; weakening, #hate &amp; . #science #cdnpoliâ¦ 
True Negative
@user is this for real? #waspi 50sborn in abject povey but we are all in it together! @user  
shit happens but life goes on. good blessðð»ðð»   days
False Positive
i literally bleed for you. don't fucking tell me you don't want my help. fuck you.   #takeitoutonme #fuck
jokes on you ð #suicidesquad #thejoker #2016 #jaredleto   
False Negative
@user my story: how #race and  have played a role in my life -  
anothr #bloated  #jerodtwin #colluder #angrygaymafia #koolaid #guzzling #hasbeen #failure #flushedð½â¦ 


---
## Fake News

We now want to create a fake news prediction mode.  We can do this using a database of articles that have already been labeled as either fake news, or real news.  To match the output of the hate speech, we'll also amend the labels from FAKE and REAL to 1 and 0.

In [7]:
fn_train = pd.read_csv("./data/fake_news.csv")
fn_train = fn_train.replace({"label" : {"FAKE" : 1, "REAL" : 0}})

print2.bold("Training Set")
print(f"Shape: {fn_train.shape}, Len: {len(fn_train)}")
print()
print2.heading("Head of training set")
print(fn_train.head())
print()
print2.bold("Article No. 2")
print2.underlined(fn_train.title.iloc[1])
print(fn_train.text.iloc[2][0:250] + "...")

Training Set
Shape: (6335, 4), Len: 6335

Head of training set
   Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text  label  
0  Daniel Greenfield, a Shillman Journalism Fello...      1  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...      1  
2  U.S. Secretary of State John F. Kerry said Mon...      0  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...      1  
4  It's primary day in New York and front-runners...      0  

Article No. 2
Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)
U.S. Secretary of State John F. Kerry said Monday that 

#### Building the model
We'll want to feed our custom model a pipeline that processes and classifies our articles. Whilst the approach is similar to our hate speech model, we'll be using a different pipeline.

1. We start the same, using a **CountVectorizer()** on our article.  This creates a matrix of 1's (appears) and 0's (doesn't appear) of one to two words.
2. We'll then apply a **TfidfTransformer()** as before to this matrix, which normalises it.  TF-IDF is an information retrieval and information extraction subtask which aims to express the importance of a word to a document which is part of a collection.
3. However, rather than applying the **SGDClassifier()**, to classify / predict each normalised matrix, we'll use a **PassiveAggresiveClassifer**, which is useful when working on large data.

In [8]:
dfn_pipeline = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
                         ('tfidf',  TfidfTransformer()),
                         ('pac',   PassiveAggressiveClassifier(max_iter=100)),])


dfn = nlp_model(dfn_pipeline)

As before, we then want to split our data into training and testing data sets, to validate our results.  However, prior to doing this, we'll want to resample our data using our custom resample function.  This function will resample all minority labels to appear the same number of times as the majority label.

We then use this to fit (and if applicable, save) our model, before predicting the accuracy of our model.

In [9]:
fn_train = dhs.resample(fn_train, "label")
fn_X_train, fn_X_test, fn_y_train, fn_y_test = train_test_split(fn_train['text'], fn_train['label'], random_state = dhs.random_state)

In [10]:
dfn.fit(fn_X_train, fn_y_train)
dfn.save_model("./models/fake_news_v1.pkl")
dfn = nlp_model()
dfn.load_model("./models/fake_news_v1.pkl")

In [11]:
fn_y_predict = dfn.predict(fn_X_test)
output_score(fn_y_test, fn_y_predict, [0, 1], accuracy_score)

filter_TP = ((fn_y_test == 1) & (fn_y_predict == 1))
filter_TN = ((fn_y_test == 0) & (fn_y_predict == 0))
filter_FP = ((fn_y_test == 0) & (fn_y_predict == 1))
filter_FN = ((fn_y_test == 1) & (fn_y_predict == 0))

print()
print2.heading("Samples of each prediction outcome")
print_samples(1, fn_X_test, filter_TP, title = "True Positive")
#print_samples(1, fn_X_test, filter_TN, title = "True Negative")
print_samples(1, fn_X_test, filter_FP, title = "False Positive")
#print_samples(1, fn_X_test, filter_FN, title = "False Negative")

Accuracy: 96.15%
Confusion Matrix
     0    1
0  773   37
1   24  752

Samples of each prediction outcome
True Positive
Here’s the timing: 
Most of the US will have to wait for polling stations to close – typically between 19:00 EST (00:00 GMT) and 20:00 EST (01:00 GMT) – for state projections. 
As for the final result? Stay glued to your phone or TV or set your alarm for 23:00 EST (04:00 GMT). That’s when West Coast polls close and history suggests a winner’s declared. It was bang on the hour in 2008, and 15 minutes later in 2012. 
Of course, if you go further back in history, 2004 was a nailbiter. I remember very well going to bed after the Kerry campaign said they’d challenge the result based on Ohio, and getting up in the morning to find out they’d caved. And of course election 2000 was what it was. 
There will be many sites tracking the results as they come in; here’s Politico’s for the presidency (they also have the House and the Senate). It’s impossible to know which one is the 

---


# 🎓 Summary
This concludes the first part of our NLP project.  Using a simple custom model class that we load in, we create two models, one that predicts if text is hate speech and another to predict if the text is fake news.

We save both trained models, to be used in our final part, where we bring all of our NLP models together.

It should be noted that neither model is too complicated. We don't validate the source of articles, nor do we provide a list of unsafe words to flag, or understand the context of the message (e.g. a reply to a tweet may not appear to be hate speech when read alone, but with the context of the original tweet, it may very well be hate speech).

That said, the models remain fairly effective because it is good at picking up the patterns that are typically seen in hate speech or fake news.